In [104]:
import pandas as pd

df = pd.read_csv('super_bowl_tweets.csv')

df.head()

,Unnamed: 0,id,created_at,verified,username,text,location
0,0,1224042733859409921,2020-02-02 18:51:44,False,pnix49,b'RT @maxcredits: Remember when the Super Bowl...,TN
1,1,1224042733712629760,2020-02-02 18:51:43,False,kevjones5,b'RT @finalfourcast: Head on over to our Insta...,"morgantown,wv"
2,2,1224042733658046465,2020-02-02 18:51:43,False,kewannadabrat,b'RT @kend0llass: this the first year i could ...,NaN
3,3,1224042733326622721,2020-02-02 18:51:43,False,tinowilliams23,b'High five \xf0\x9f\x99\x8b\xf0\x9f\x8f\xbd\x...,NaN
4,4,1224042733314224129,2020-02-02 18:51:43,False,ThisIsAnch,b'I can\xe2\x80\x99t wait until this day next ...,"Jacksonville, FL"


In [105]:
def decode_string(s):
    return s.lstrip('b"').lstrip("b'")

df['text'] = df['text'].map(decode_string)

In [106]:
df['text']

def is_retweet(s):
    if s.startswith('RT '):
        return True
    else:
        return False
    

df['retweet'] = df['text'].map(is_retweet)
df.head()

,Unnamed: 0,id,created_at,verified,username,text,location,retweet
0,0,1224042733859409921,2020-02-02 18:51:44,False,pnix49,RT @maxcredits: Remember when the Super Bowl w...,TN,True
1,1,1224042733712629760,2020-02-02 18:51:43,False,kevjones5,RT @finalfourcast: Head on over to our Instagr...,"morgantown,wv",True
2,2,1224042733658046465,2020-02-02 18:51:43,False,kewannadabrat,RT @kend0llass: this the first year i could ca...,NaN,True
3,3,1224042733326622721,2020-02-02 18:51:43,False,tinowilliams23,High five \xf0\x9f\x99\x8b\xf0\x9f\x8f\xbd\xe2...,NaN,False
4,4,1224042733314224129,2020-02-02 18:51:43,False,ThisIsAnch,I can\xe2\x80\x99t wait until this day next ye...,"Jacksonville, FL",False


In [107]:
def is_mention(s):
    if '@' in s.lstrip('RT @'):
        return True
    else:
        return False
    
df['mention'] = df['text'].map(is_mention)

def is_reply(s):
    if s.startswith('@'):
        return True
    else:
        return False
    
df['reply'] = df['text'].map(is_reply)

In [108]:
df.head()

,Unnamed: 0,id,created_at,verified,username,text,location,retweet,mention,reply
0,0,1224042733859409921,2020-02-02 18:51:44,False,pnix49,RT @maxcredits: Remember when the Super Bowl w...,TN,True,False,False
1,1,1224042733712629760,2020-02-02 18:51:43,False,kevjones5,RT @finalfourcast: Head on over to our Instagr...,"morgantown,wv",True,True,False
2,2,1224042733658046465,2020-02-02 18:51:43,False,kewannadabrat,RT @kend0llass: this the first year i could ca...,NaN,True,False,False
3,3,1224042733326622721,2020-02-02 18:51:43,False,tinowilliams23,High five \xf0\x9f\x99\x8b\xf0\x9f\x8f\xbd\xe2...,NaN,False,False,False
4,4,1224042733314224129,2020-02-02 18:51:43,False,ThisIsAnch,I can\xe2\x80\x99t wait until this day next ye...,"Jacksonville, FL",False,False,False


In [109]:
import re
mentioners = []
mentionees = []
repliers = []
replied_to = []
retweeters = []
retweetees = []

def get_ats(s):
    s = s.lstrip('RT @').lstrip('@')
    for user in re.findall('@[A-Za-z]+', s):
        mentionees.append(user.lstrip('@'))
        mentioners.append(df['username'][i])
        
def get_replied(s):
    s = s.lstrip('@')
    return re.split(' ',s)[0]

for i in df.index.values:
    if df['retweet'][i]:
        retweeters.append(df['username'][i])
        retweetees.append(df['text'][i].lstrip('RT @'))
    if df['mention'][i]:
        get_ats(df['text'][i])
    if df['reply'][i]:
        replied_to.append(get_replied(df['text'][i]))
        repliers.append(df['username'][i])
        

        

In [110]:
df_network = pd.DataFrame()

In [111]:
df_network = pd.DataFrame()
mentioners = pd.Series(mentioners)
mentionees = pd.Series(mentionees)
df_network = pd.DataFrame([mentioners, mentionees])
df_network = df_network.T
df_network['Type'] = pd.Series(['Mention']*len(mentioners))
df_network.head()

,0,1,Type
0,kevjones5,TheDaSeanButler,Mention
1,kevjones5,kevjones,Mention
2,kevjones5,AshleyRockRadio,Mention
3,Darioescsilva,shakira,Mention
4,rokinrhonda,sunflowermoon,Mention


In [112]:
retweeters = pd.Series(retweeters)
retweetees = pd.Series(retweetees)
df_network_retweet = pd.DataFrame([retweeters, retweetees])
df_network_retweet = df_network_retweet.T
df_network_retweet['Type'] = pd.Series(['Retweet']*len(retweeters))
df_network_retweet.head()

,0,1,Type
0,pnix49,maxcredits: Remember when the Super Bowl was a...,Retweet
1,kevjones5,finalfourcast: Head on over to our Instagram t...,Retweet
2,kewannadabrat,kend0llass: this the first year i could care l...,Retweet
3,BlGCHUNGUS,pissboymcgee: today is the day fellas. our day...,Retweet
4,AllforTrump8,hrkbenowen: \xe2\x80\xa6\xe2\x80\xa6\xe2\x80\x...,Retweet


In [113]:
repliers = pd.Series(repliers)
replied_to = pd.Series(replied_to)
df_network_reply = pd.DataFrame([repliers, replied_to])
df_network_reply = df_network_reply.T
df_network_reply['Type'] = pd.Series(['Reply']*len(repliers))
df_network_reply.head()

,0,1,Type
0,allenius1,BlackScapeRose,Reply
1,Imagecaptured,catturd2,Reply
2,zlilzzz,ImStillDontai,Reply
3,ericsuniverse,LeandroPiromal,Reply
4,houseplant1111,dril,Reply


In [114]:
dfs = [df_network, df_network_retweet, df_network_reply]

df_network = pd.concat(dfs, ignore_index = True)

In [115]:
df_network

,0,1,Type
0,kevjones5,TheDaSeanButler,Mention
1,kevjones5,kevjones,Mention
2,kevjones5,AshleyRockRadio,Mention
3,Darioescsilva,shakira,Mention
4,rokinrhonda,sunflowermoon,Mention
5,rokinrhonda,AnastasiaRenae,Mention
6,rokinrhonda,JWat,Mention
7,rokinrhonda,deed,Mention
8,sonicmoremusic1,YouTube,Mention
9,SidInTheCut,NFLonFOX,Mention


In [116]:
df_network['Type'].value_counts()

Retweet    3438
Mention    1145
Reply       252
Name: Type, dtype: int64

In [117]:
len(df)

5000